## Prompt Engineering

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
llm = ChatOpenAI(temperature=0.3,
                 openai_api_key=OPENAI_API_KEY,
                 model_name='gpt-3.5-turbo-0613',
                )

In [10]:
def generate_response(llm: ChatOpenAI, quant_topic: str, quant_title: str) -> tuple[str,str]:
    """Generate AI Twitter Content for QuantPy Media Channels

    Parameters:
        - llm:  pre-trained ChatOpenAi large language model
        - quant_topic: Topic in Quant Finance
        - quant_topic: Topic in Quant Finance

    Returns:
        - tuple[long response,short reposonse]: Chat GPT long and short responses
    """
    # System Template for LLM to follow
    system_template = """
        You are an incredibly wise and smart quantitative analyst that lives and breathes the world of quantitative finance.
        Your goal is to writing short-form content for twitter given a `topic` in the area of quantitative finance and a `title` from the user.
        
        % RESPONSE TONE:

        - Your response should be given in an active voice and be opinionated
        - Your tone should be serious w/ a hint of wit and sarcasm
        
        % RESPONSE FORMAT:
        
        - Be extremely clear and concise
        - Respond in short phrases
        - No longer than 40 words total
        - Total of 280 characters (counting spaces and other characters)
        - Do not respond with emojis
        
        % RESPONSE CONTENT:

        - Include specific examples of where this is used in the quantitative finance space
        - If you don't have an answer, say, "Sorry, I'll have to ask the Quant Finance Gods!"    

        % RESPONSE TEMPLATE:

        - Here’s a condensed structure tailored for Twitter's 280-character limit: 
            Hook: Captivate with a one-liner.
            Intro: Briefly introduce the topic.
            Explanation: Simplify the core idea.
            Application: Note real-world relevance.
            Closing: Reflective one-liner.
            Action: Short engagement call.
            Engagement: Quick question.
    
    """
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    # human template for input
    human_template="topic to write about is {topic}, and the title will be {title}. Keep the total response under 200 characters total! respond in short phrases, only one sentence maximium per line no more than 6 words"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template, 
                                                                    input_variables=["topic", "title"])

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, 
                                                    human_message_prompt])

    # get a chat completion from the formatted messages
    final_prompt = chat_prompt.format_prompt(topic=quant_topic, 
                                             title=quant_title).to_messages()
    first_response = llm(final_prompt).content

    ai_message_prompt = AIMessagePromptTemplate.from_template(first_response)

    # reminder of length
    reminder_template="This was good, but way too long, please make your response much more concise and much shorter! Please maintain the existing template."
    reminder_prompt = HumanMessagePromptTemplate.from_template(reminder_template)

    chat_prompt2 = ChatPromptTemplate.from_messages([system_message_prompt, 
                                                     human_template, 
                                                     ai_message_prompt, 
                                                     reminder_prompt])

    # get a chat completion from the formatted messages
    final_prompt = chat_prompt2.format_prompt(topic=quant_topic, 
                                              title=quant_title).to_messages()
    short_response = llm(final_prompt).content

    return first_response, short_response

In [11]:
first_response, short_response = generate_response(llm, quant_topic='Time Value of Money', quant_title='Unveiling the Magic of Compounding: Time Value of Money')

In [14]:
count_length = lambda d: sum(len(d[val]) for val in d)
key_list=["Hook", "Intro", "Explanation", "Application", "Closing", "Action", "Engagement"]
def extract_tweet(openai_tweet: str, key_list: list) -> dict:
    """Creates dictionary from Openai response using keyword template

    Parameters:
        - openai_tweet: 
        - key_list: list key words used for searching reponse template

    Returns:
        - dictionary: templated tweet
    """
    # Iterate through key list
    for i, key in enumerate(key_list):
        # find starting position
        start = openai_tweet.find(key_list[i])+len(key_list[i])+2
        if i != len(key_list) - 1:
            # using ending position, subset str and append to template
            end = openai_tweet.find(key_list[i+1])
            line = openai_tweet[start:end]
            template[key_list[i]] = line
        else:
            # if final word in list, only subsection by start word
            template[key_list[i]] = openai_tweet[start:]
    return template

In [215]:
extract_tweet(first_response)

({'Hook': 'Discover the enchantment of compounding!\n',
  'Intro': 'Unveiling the Time Value of Money.\n',
  'Explanation': 'Money grows exponentially over time.\n',
  'Application': 'Investing, loans, retirement planning, and more.\n',
  'Closing': 'Embrace the power of compounding!\n',
  'Action': 'Start investing today!\n',
  'Engagement': 'How has compounding changed your financial outlook?'},
 270)

In [216]:
extract_tweet(short_response)

({'Hook': 'Unveiling the magic of compounding!\n',
  'Intro': 'Time Value of Money explained.\n',
  'Explanation': 'Money grows exponentially over time.\n',
  'Application': 'Investing, loans, retirement planning, and more.\n',
  'Closing': 'Embrace the power of compounding!\n',
  'Action': 'Start investing today!\n',
  'Engagement': 'How has compounding changed your finances?'},
 252)

In [65]:
# Preprocessing content-ideas

In [82]:
file = open("content-ideas.txt", "r")
quant_tweets = {}

for line_no, line in enumerate(file.readlines()):
    # start 2nd row to avoid heading and underlines
    if line_no > 1:
        # split on line divisors
        items = line.split('|')
        # capture and ensure int and str formatting
        tweet_no = int(items[1])
        quant_topic = items[2].strip()
        quant_title = items[3].strip().strip('"')
        # store within dict
        quant_tweets[tweet_no] = {}
        quant_tweets[tweet_no]['topic'] = quant_topic
        quant_tweets[tweet_no]['title'] = quant_title
        # print tweet no, topic and title
        print(f"{tweet_no}_{quant_topic}_{quant_title}")
file.close()

# storing in desired format
with open('quants_tweets.txt', 'w') as f:
    for tweet_no, tweet_info in quant_tweets.items():
        tweet_repr = str(tweet_no)+'|'+tweet_info['topic']+'|'+tweet_info['title']+'|FALSE|\n'
        f.write(tweet_repr)

1_Time Value of Money_Unveiling the Magic of Compounding: Time Value of Money
2_Risk and Return_Playing the Odds: Understanding Risk and Return
3_Modern Portfolio Theory_Crafting the Perfect Portfolio: An Intro to MPT
4_Black-Scholes Model_The Black-Scholes Legacy: Revolutionizing Option Pricing
5_Multifactor Models_Beyond Beta: Exploring Multifactor Models
6_Copula Models_Bridging Dependencies: An Insight into Copula Models
7_Stochastic Calculus_The Dance of Chance: Delving into Stochastic Calculus
8_Ito's Lemma_Ito's Insight: Unpacking Ito's Lemma
9_Quantitative Risk Management_Taming Uncertainty: Quantitative Risk Management Essentials
10_Simple Moving Average Strategy_Smooth Moves: SMA Trading Strategy Explained
11_Backtesting_Rewinding The Market: The Art of Backtesting
12_Transaction Costs and Slippage_The Hidden Costs: Navigating Transaction Costs and Slippage
13_Portfolio Optimization_Balancing Act: Portfolio Optimization Techniques
14_QuantLib Introduction_Toolkit Talk: A Dive

In [360]:
from enum import Flag
from collections import deque
from dataclasses import dataclass, field

class Boolean(Flag):
    TRUE = True
    FALSE = False

class TweetDeque(deque):
    """
    Container for processing, sorting Tweet dataclass objects
    """
    def sort(self, *args, **kwargs):        
        items = [self.pop() for x in range(len(self))]
        items.sort(*args, **kwargs)
        self.extend(items)

@dataclass
class Tweet:
    """Class for keeping track of Tweets"""
    id: int
    topic: str
    title: str
    sent_status: Boolean
    tweet: str = field(default="")

    def __lt__(self, other):
        return (
            self.sent_status.value, self.id
        ) < (
            other.sent_status.value, other.id
        )

    @classmethod
    def from_str(cls, tweet_line: str):
        # underscores used to indicate unpacked variables, only used internally
        _id, _topic, _title, _sent_status, _next_line = tweet_line.split('|')
        # convert status TRUE/FALSE to Enum Representation
        _sent_status_bool=Boolean.TRUE if _sent_status == Boolean.TRUE.name else Boolean.FALSE
        # return class
        return cls(
            id=int(_id),
            topic=_topic,
            title=_title,
            sent_status=_sent_status_bool
        )

    def to_str(self):
        return f"{self.id}|{self.topic}|{self.title}|{self.sent_status.name}|\n"

    def update_status(self, new_status: Boolean):
        self.sent_status = new_status
    
@dataclass
class TweetQueue:
    tweets: list[Tweet] = field(default_factory=TweetDeque)
    tweets_not_sent: list[Tweet] = field(init=False)

    def __post_init__(self):
        self.tweets_not_sent = self.list_not_sent()

    def __len__(self):
        return len(self.tweets)

    def __iter__(self):
        yield from self.tweets

    def enqueue(self, tweet):
        # print(f"{tweet.to_str()} will be added.")
        self.tweets.append(tweet)

    def dequeue(self):
        # print(f"{self.tweets[0].to_str()} will be removed.")
        return self.tweets.popleft()

    def list_not_sent(self):
        _tweets_not_sent = [tweet for tweet in self.tweets if not tweet.sent_status]
        print(_tweets_not_sent)
        return _tweets_not_sent
        
    @classmethod
    def from_text_file(cls, text_file):
        _tweets = cls()
        for tweet_line in open(text_file, 'r'):
            tweet = Tweet.from_str(tweet_line)
            _tweets.enqueue(tweet)
        return _tweets

    def to_text_file(self, text_file):
        with open(text_file, 'w') as f:
            for tweet in self.tweets:
                tweet_line = tweet.to_str()
                f.write(tweet_line)
        

In [361]:
tweetQueue = TweetQueue.from_text_file('quants_tweets.txt')
tweetQueueCheck = tweetQueue.to_text_file('quants_tweets_check.txt')
# tweetQueueVerify = TweetQueue.from_text_file('quants_tweets_check.txt')

[]


In [357]:
tweetQueue == tweetQueueVerify
print(len(tweetQueue))
tweetQueue.tweets[0]
tweetQueue.tweets[0].update_status(Boolean.TRUE)
print(len(tweetQueue))

100
100


In [352]:
quant_tweet_idea = tweetQueue.tweets_not_sent[0]
first_response, short_response = generate_response(llm, quant_topic=quant_tweet_idea.topic, quant_title=quant_tweet_idea.title)
first_draft = extract_tweet(first_response)
short_response = extract_tweet(short_response)

IndexError: list index out of range

In [344]:
list_not_sent = TweetQueue.list_not_sent
list_not_sent

In [329]:
quant_tweet_idea = TweetQueue.list_not_sent:
    first_response, short_response = generate_response(llm, quant_topic=quant_tweet_idea.topic, quant_title=quant_tweet_idea.title)
    first_draft = extract_tweet(first_response)
    short_response = extract_tweet(short_response)

SyntaxError: invalid syntax (2576123844.py, line 1)

In [37]:
from queues import Queue

fifo = Queue("1st", "2nd", "3rd")
len(fifo)

3

In [41]:
for i in fifo:
    print(i)
    print(fifo)

1st
2nd
3rd
